# IMPORT LIBARAY

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import datetime
from collections import Counter
from scipy import interpolate

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import plotly.graph_objects as go
from matplotlib import dates
from plotly.subplots import make_subplots
import plotly
import plotly.express as px
pd.options.display.float_format = '{:.5f}'.format

import FinanceDataReader as fdr

from sklearn.preprocessing import MinMaxScaler

# os.getcwd()
### 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
font_list
plt.rc("font", family="DejaVu Sans")
parameters = {'axes.labelsize': 20,
          'axes.titlesize': 25,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
        'legend.fontsize': 20,
          }
plt.rcParams.update(parameters)
plt.rcParams['font.family'] = 'DejaVu Sans'

current_dir = os.getcwd()
plt.style.use("seaborn-dark")
plt.rcParams['font.family'] = 'DejaVu Sans'

In [ ]:
root_path = '../../../'
current_dir = os.getcwd()

master_path = os.path.join(root_path, '01. Data','00. master_data')
rawfile_path = os.path.join(master_path, "result_xlsx")

out_path = os.path.join(root_path, '01. Data','01. 재무지표1', '02. preprocessed')

# 결측처리 및 컬럼 처리

In [3]:
desc = pd.read_excel(os.path.join(master_path, "desc_v2.2.xlsx"), sheet_name='desc', header=0, usecols=['no','raw','variable','group','importance'])
desc.head(2)

,raw,no,variable,group,importance
0,Date_x,1,Date_x,DT,9
1,BS_Earnings Quality Score,2,BS_Earnings Quality Score,BS,5


In [4]:
# desc 데이터 기준, group이 한글인 경우(재무팀에서 뽑아준 항목들)만 target variable로 가져감
target_variable = desc.loc[~desc['group'].apply(lambda x: x.encode().isalpha()),"variable"].values.tolist()

dic_group = dict()
for _, xcol in enumerate(desc['variable'].values):
    dic_group[xcol] = desc.loc[_,'group']

dic_imp = dict()
for _, xcol in enumerate(desc['variable'].values):
    dic_imp[xcol] = desc.loc[_,'importance']
    
dic_name = dict()
for _, xcol in enumerate(desc['raw'].values):
    dic_name[xcol] = desc.loc[_, 'variable']
    
Y = ['close','close_weighted','EV_EBITDA','PBR'] # Y1, Y2, Y4, Y3

## Accenture

* 초기 4개 행 제거(2002/8, 2002/11, 2003/2, 2003/5)
* "자본금(천원)"컬럼 삭제(이유: 모두 0의 값을 가짐)

<br>
* desc 2.2적용 코드

In [42]:
raw_file_name = "Accenture_non_shift.xlsx"
# Acc
# TCS

df = pd.read_excel(rawfile_path+raw_file_name)
df['Date_x'] = pd.to_datetime(df['Date_x'])
df.head(2)

,Date_x,BS_Earnings Quality Score,BS_Cash and Short Term Investments,BS_Cash & Equivalents,BS_Short Term Investments,"BS_Accounts Receivable - Trade, Net","BS_Accounts Receivable - Trade, Gross",BS_Provision for Doubtful Accounts,"BS_Total Receivables, Net","BS_Other Current Assets, Total",...,Opex Margin,Y1,Y2,Y3,Y4,인당매출s,무형자산회전율s,인당영업이익s,인당순이익s,투자비 대비 매출 (Rev/GI)s
0,2002-08-01,94,1317,1317,0,2144,1703,-41,2144,591,...,0.08395,22.87444,19.58500,34.93000,3.59000,NaN,NaN,NaN,NaN,NaN
1,2002-11-01,94,1474,1474,0,2291,1703,-41,2291,511,...,0.12894,16.91222,16.61000,30.90000,4.01000,NaN,NaN,NaN,NaN,NaN


In [43]:
for col in df.columns:
    if col in list(dic_name.keys()) :
#         print(col, dic_name[col])
        df.rename(columns={col:dic_name[col]}, inplace=True)
new_df = df.loc[4:, ["Date_x"]+list(set(df.columns)&set(target_variable))+Y].reset_index(drop=True)
print(new_df.iloc[0,:].isna().sum())
print(new_df.iloc[-1,:].isna().sum())

0
0


In [44]:
tmp = new_df.describe().loc['std',:][new_df.describe().loc['std',:] == 0].index
TF = new_df.describe().loc['min',:] == new_df.describe().loc['max',:]
tmp1 = new_df.describe().loc[:,TF].columns.values.tolist()

tmp = list(set(tmp) | set(tmp1))
tmp = list(set(tmp) & set(new_df.columns))

drop_lst = list(tmp)

print("before: ", new_df.shape)
print("drop_lst: ",len(list(drop_lst)))
new_df = new_df[new_df.columns.difference(drop_lst)]
print("after: ", new_df.shape)

drop_lst


before:  (76, 60)
drop_lst:  1
after:  (76, 59)


['자본금(천원)']

In [45]:
print("결측값 확인: ", new_df.isna().sum().sum())

결측값 확인:  0


In [36]:
new_df[new_df.isna().sum()[new_df.isna().sum()>0].index]

""
4
5
6
7
8
...
75
76
77
78


In [46]:
for i in range(len(new_df.columns)):
    flg = 0
    for j in range(len(new_df)):
        if pd.isna(new_df.iloc[j,i]):
            flg += 1
        else:
            flg = 0
            
        if flg >= 4:
            print(new_df.columns[i])
            flg = 0
            break
print("4개 이상의 결측값을 가지는 컬럼 확인 ")

4개 이상의 결측값을 가지는 컬럼 확인 


In [47]:
# 선형 보간
print("결측값 확인(before): ", new_df.isna().sum().sum())
for _, col in enumerate(new_df.columns):
    if col != "Date_x":
        new_df[col] = new_df[col].interpolate(method='linear')

print("선형 보간 적용")
print("결측값 확인(after): ", new_df.isna().sum().sum())

선형 보간 적용
결측값 확인:  0


In [48]:
# 이 때, 편의를 위해 "Date_x" 변수를 맨 앞으로, Y1~Y4 변수는 맨 뒤로 위치시킴
col1 = new_df.pop("Date_x")
new_df.insert(0, col1.name, col1)

col2 = new_df.pop("close")
new_df.insert(new_df.shape[1], col2.name, col2)

col3 = new_df.pop("close_weighted")
new_df.insert(new_df.shape[1], col3.name, col3)

col4 = new_df.pop("EV_EBITDA")
new_df.insert(new_df.shape[1], col4.name, col4)

col5 = new_df.pop("PBR")
new_df.insert(new_df.shape[1], col5.name, col5)

print("new_df.shape: ",new_df.shape)


new_df.head(2)

new_df.shape:  (76, 59)


,Date_x,"BPS(지배, Adj.)(원/주)",BPS증가율(YoY)(비율),CAPEX(천원),EBIT(천원),EBITDA2(천원),EBITDA2마진율(비율),EBITDA2증가율(YoY)(비율),"EPS(지배, Adj.)(원/주)",EPS증가율(YoY)(비율),...,자산총계(천원),재고자산회전율(비율),채무총계,총자산회전율(비율),투자비 대비 매출 (Rev/GI)s,현금배당성향(%),close,close_weighted,EV_EBITDA,PBR
0,2003-08-01,14.20292,-0.41881,76.28000,345,403.06000,13.36000,0.22854,0.25000,2.12500,...,6459,0.68731,60,1.98000,-49.33851,0.00000,25.11197,25.70000,4.37000,24.02000
1,2003-11-01,11.97411,-0.11801,44.50000,421,483.45000,14.82000,-0.01614,0.33000,0.22222,...,6703,0.68050,50,1.97000,-32.43158,0.00000,23.99435,22.45000,6.31000,22.48000


### 파일 내보내기

In [49]:
### 최종 dataset을 csv, xlsx 파일로 out ###
new_df.to_csv(os.path.join(out_path,"ITS")+raw_file_name.split('.')[0]+"_clean_v1.3.csv", index=False, encoding='utf-8')
new_df.to_excel(os.path.join(out_path,"ITS")+raw_file_name.split('.')[0]+"_clean_v1.3.xlsx", index=False)

## TCS

* 초기 6개의 행 삭제(2005/6, 2005/9, 2005/12, 2006/3, 2006/6, 2006/9)
* 마지막 행 삭제(2022/6, 이유: Y3, Y4를 포함한 다수의 컬럼에서 결측값 존재)
* 이외의 결측값은 선형 보간법 적용

In [17]:
raw_file_name = "Tcs_non_shift.xlsx"
# Acc
# TCS

df = pd.read_excel(rawfile_path+raw_file_name)
df['Date_x'] = pd.to_datetime(df['Date_x'])
df.head(2)

,Date_x,BS_Earnings Quality Score,BS_Period End FX Rate (INR/USD),BS_Cash and Short Term Investments,BS_Cash,BS_Cash & Equivalents,BS_Short Term Investments,"BS_Accounts Receivable - Trade, Net","BS_Accounts Receivable - Trade, Gross",BS_Provision for Doubtful Accounts,...,Opex Margin,Y1,Y2,Y3,Y4,인당매출s,무형자산회전율s,인당영업이익s,인당순이익s,투자비 대비 매출 (Rev/GI)s
0,2005-06-01,99,0.02301,87,77,176,10,593,518,-16,...,0.55004,3.72412,3.20328,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-09-01,99,0.02276,84,75,176,9,686,611,-18,...,0.69203,3.42314,3.77240,3.85000,19.44000,NaN,NaN,NaN,NaN,NaN


In [19]:
for col in df.columns:
    if col in list(dic_name.keys()) :
#         print(col, dic_name[col])
        df.rename(columns={col:dic_name[col]}, inplace=True)
    
# new_df = df.loc[5:len(df)-2, ["Date_x"]+list(set(df.columns)&set(target_variable))+Y].reset_index(drop=True)
new_df = df.loc[6:len(df)-2, ["Date_x"]+list(set(df.columns)&set(target_variable))+Y].reset_index(drop=True)
print(new_df.iloc[0,:].isna().sum())
print(new_df.iloc[-1,:].isna().sum())

0
0


In [10]:
tmp = new_df.describe().loc['std',:][new_df.describe().loc['std',:] == 0].index
TF = new_df.describe().loc['min',:] == new_df.describe().loc['max',:]
tmp1 = new_df.describe().loc[:,TF].columns.values.tolist()

tmp = list(set(tmp) | set(tmp1))
tmp = list(set(tmp) & set(new_df.columns))

drop_lst = list(tmp)

print("before: ", new_df.shape)
print("drop_lst: ",len(list(drop_lst)))
new_df = new_df[new_df.columns.difference(drop_lst)]
print("after: ", new_df.shape)

drop_lst


before:  (61, 60)
drop_lst:  0
after:  (61, 60)


[]

In [11]:
for i in range(len(new_df.columns)):
    flg = 0
    for j in range(len(new_df)):
        if pd.isna(new_df.iloc[j,i]):
            flg += 1
        else:
            flg = 0
            
        if flg >= 4:
            print(new_df.columns[i])
            flg = 0
            break
print("4개 이상의 결측값을 가지는 컬럼 확인 ")

4개 이상의 결측값을 가지는 컬럼 확인 


In [12]:
print("결측값 확인: ", new_df.isna().sum().sum())

결측값 확인:  10


In [13]:
# 결측값을 가지는 컬럼들 확인
new_df[new_df.isna().sum()[new_df.isna().sum()>0].index]

,ROGI(%),ROE(지배)(비율),유동비율(비율),CAPEX(천원),ROA(비율),총자산회전율(비율)
0,0.65803,59.28000,2.17000,49.45000,41.56000,1.88000
1,0.59256,56.74000,2.22000,90.29000,39.02000,1.71000
2,0.54499,55.10000,1.99000,60.73000,37.46000,1.61000
3,0.52015,NaN,2.42000,62.88000,NaN,NaN
4,0.34804,47.75000,2.24000,116.44000,32.89000,1.47000
...,...,...,...,...,...,...
56,0.58123,38.03000,2.91000,159.38000,25.88000,1.30000
57,0.61288,38.32000,2.93000,74.05000,26.30000,1.30000
58,0.62835,38.08000,3.15000,96.67000,26.62000,1.29000
59,0.80827,40.94000,3.30000,121.47000,25.85000,1.26000


In [14]:
# 선형 보간
print("결측값 확인(before): ", new_df.isna().sum().sum())
for _, col in enumerate(new_df.columns):
    if col != "Date_x":
        new_df[col] = new_df[col].interpolate(method='linear')

print("선형 보간 적용")
print("결측값 확인(after): ", new_df.isna().sum().sum())

결측값 확인(before):  10
선형 보간 적용
결측값 확인(after):  0


In [15]:
# 이 때, 편의를 위해 "Date_x" 변수를 맨 앞으로, Y1~Y4 변수는 맨 뒤로 위치시킴
col1 = new_df.pop("Date_x")
new_df.insert(0, col1.name, col1)

col2 = new_df.pop("close")
new_df.insert(new_df.shape[1], col2.name, col2)

col3 = new_df.pop("close_weighted")
new_df.insert(new_df.shape[1], col3.name, col3)

col4 = new_df.pop("EV_EBITDA")
new_df.insert(new_df.shape[1], col4.name, col4)

col5 = new_df.pop("PBR")
new_df.insert(new_df.shape[1], col5.name, col5)

print("new_df.shape: ",new_df.shape)


new_df.head(2)

new_df.shape:  (61, 60)


,Date_x,EBITDA2증가율(YoY)(비율),이자보상율,ROGI(%),ROE(지배)(비율),이익잉여금(천원),유동비율(비율),SG&A,부채총계(천원),부채비율(비율),...,무형자산,운전자본증감(천원),EBITDA2(천원),인당순이익s,순이익률(비율),자산총계(천원),close,close_weighted,EV_EBITDA,PBR
0,2006-12-01,0.43921,95000.00000,0.65803,59.28000,1350,2.17000,731.50000,840,45.65217,...,70,538.60000,309.92000,0.01377,23.20000,2680,4.96502,5.25436,24.91000,7.11000
1,2007-03-01,0.44910,3725.92593,0.59256,56.74000,1559,2.22000,554.80000,1006,49.00146,...,70,769.90000,332.93000,0.01374,23.41000,3059,5.97507,5.02981,23.31000,6.59000


### 파일 내보내기

In [16]:
### 최종 dataset을 csv, xlsx 파일로 out ###
new_df.to_csv(os.path.join(out_path,"ITS")+raw_file_name.split('.')[0]+"_clean_v1.4.csv", index=False, encoding='utf-8')
new_df.to_excel(os.path.join(out_path,"ITS")+raw_file_name.split('.')[0]+"_clean_v1.4.xlsx", index=False)

## Cognizant

* 초기 4개의 행 삭제(2002/6, 2002/9, 2002/12, 2003/3)

In [121]:
raw_file_name = "Cognizant_non_shift.xlsx"
# Acc
# TCS

df = pd.read_excel(rawfile_path+raw_file_name)
df['Date_x'] = pd.to_datetime(df['Date_x'])
df.head(2)

,Date_x,BS_Earnings Quality Score,BS_Cash and Short Term Investments,BS_Cash & Equivalents,BS_Short Term Investments,"BS_Accounts Receivable - Trade, Net","BS_Accounts Receivable - Trade, Gross",BS_Provision for Doubtful Accounts,"BS_Total Receivables, Net",BS_Total Inventory,...,Opex Margin,Y1,Y2,Y3,Y4,인당매출s,무형자산회전율s,인당영업이익s,인당순이익s,투자비 대비 매출 (Rev/GI)s
0,2002-06-01,96,104,104,0,37,29,-1,37,1,...,0.22794,2.68322,2.30477,3.68000,7.39000,NaN,NaN,NaN,NaN,NaN
1,2002-09-01,96,123,123,0,39,31,-1,39,1,...,0.23039,2.66595,2.62974,3.52000,7.54000,NaN,NaN,NaN,NaN,NaN


In [122]:
for col in df.columns:
    if col in list(dic_name.keys()) :
#         print(col, dic_name[col])
        df.rename(columns={col:dic_name[col]}, inplace=True)
    
new_df = df.loc[4:, ["Date_x"]+list(set(df.columns)&set(target_variable))+Y].reset_index(drop=True)
print(new_df.iloc[0,:].isna().sum())
print(new_df.iloc[-1,:].isna().sum())

0
0


In [123]:
tmp = new_df.describe().loc['std',:][new_df.describe().loc['std',:] == 0].index
TF = new_df.describe().loc['min',:] == new_df.describe().loc['max',:]
tmp1 = new_df.describe().loc[:,TF].columns.values.tolist()

tmp = list(set(tmp) | set(tmp1))
tmp = list(set(tmp) & set(new_df.columns))

drop_lst = list(tmp)

print("before: ", new_df.shape)
print("drop_lst: ",len(list(drop_lst)))
new_df = new_df[new_df.columns.difference(drop_lst)]
print("after: ", new_df.shape)

drop_lst


before:  (76, 60)
drop_lst:  0
after:  (76, 60)


[]

In [124]:
for i in range(len(new_df.columns)):
    flg = 0
    for j in range(len(new_df)):
        if pd.isna(new_df.iloc[j,i]):
            flg += 1
        else:
            flg = 0
            
        if flg >= 4:
            print(new_df.columns[i])
            flg = 0
            break
print("4개 이상의 결측값을 가지는 컬럼 확인 ")

4개 이상의 결측값을 가지는 컬럼 확인 


In [125]:
print("결측값 확인: ", new_df.isna().sum().sum())
new_df[new_df.isna().sum()[new_df.isna().sum()>0].index]

결측값 확인:  0


""
0
1
2
3
4
...
71
72
73
74


In [126]:
# 선형 보간
print("결측값 확인(before): ", new_df.isna().sum().sum())
for _, col in enumerate(new_df.columns):
    if col != "Date_x":
        new_df[col] = new_df[col].interpolate(method='linear')

print("선형 보간 적용")
print("결측값 확인(after): ", new_df.isna().sum().sum())

결측값 확인(before):  0
선형 보간 적용
결측값 확인(after):  0


In [127]:
# 이 때, 편의를 위해 "Date_x" 변수를 맨 앞으로, Y1~Y4 변수는 맨 뒤로 위치시킴
col1 = new_df.pop("Date_x")
new_df.insert(0, col1.name, col1)

col2 = new_df.pop("close")
new_df.insert(new_df.shape[1], col2.name, col2)

col3 = new_df.pop("close_weighted")
new_df.insert(new_df.shape[1], col3.name, col3)

col4 = new_df.pop("EV_EBITDA")
new_df.insert(new_df.shape[1], col4.name, col4)

col5 = new_df.pop("PBR")
new_df.insert(new_df.shape[1], col5.name, col5)

print("new_df.shape: ",new_df.shape)


new_df.head(2)

new_df.shape:  (76, 60)


,Date_x,이자보상율,"EPS(지배, Adj.)(원/주)",매입채무회전율,재고자산회전율(비율),당기순이익(천원),유동자산(천원),매출총이익(천원),유동비율(비율),인당매출s,...,EBITDA2(천원),현금배당성향(%),자기자본비율,EBITDA2마진율(비율),CAPEX(천원),BPS증가율(YoY)(비율),close,close_weighted,EV_EBITDA,PBR
0,2003-06-01,219.23077,0.03000,6.74286,0.42843,13.50000,204,40.20000,4.90000,0.03832,...,19.90000,0.00000,74.90775,22.75000,7.65000,1.63109,5.51054,3.56562,20.49000,7.48000
1,2003-09-01,247.43590,0.03000,8.83333,0.40370,16.00000,243,45.10000,4.90000,0.03901,...,22.28000,0.00000,75.47771,22.71000,6.56000,1.92666,6.19883,5.26875,28.56000,9.79000


### 파일 내보내기

In [128]:
### 최종 dataset을 csv, xlsx 파일로 out ###
new_df.to_csv(os.path.join(out_path,"ITS")+raw_file_name.split('.')[0]+"_clean_v1.3.csv", index=False, encoding='utf-8')
new_df.to_excel(os.path.join(out_path,"ITS")+raw_file_name.split('.')[0]+"_clean_v1.3.xlsx", index=False)

## Infosys

* 초기 4개의 행 삭제(2002/6, 2002/9, 2002/12, 2003/3)
* 이외의 결측값은 선형 보간법 적용

In [147]:
raw_file_name = "Infosys_non_shift.xlsx"
# Acc
# TCS

df = pd.read_excel(rawfile_path+raw_file_name)
df['Date_x'] = pd.to_datetime(df['Date_x'])
df.head(2)

,Date_x,BS_Earnings Quality Score,BS_Period End FX Rate (INR/USD),BS_Cash and Short Term Investments,BS_Cash & Equivalents,BS_Short Term Investments,"BS_Accounts Receivable - Trade, Net","BS_Accounts Receivable - Trade, Gross",BS_Provision for Doubtful Accounts,"BS_Total Receivables, Net",...,Opex Margin,Y1,Y2,Y3,Y4,인당매출s,무형자산회전율s,인당영업이익s,인당순이익s,투자비 대비 매출 (Rev/GI)s
0,2002-06-01,92,0.02049,183,183,1516,85,307,-4,233,...,0.35806,1.40519,1.20477,0.22000,NaN,NaN,NaN,NaN,NaN,NaN
1,2002-09-01,92,0.02070,208,208,1516,95,307,-4,284,...,0.36419,0.99021,1.44987,0.22000,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
for col in df.columns:
    if col in list(dic_name.keys()) :
#         print(col, dic_name[col])
        df.rename(columns={col:dic_name[col]}, inplace=True)
    
new_df = df.loc[4:, ["Date_x"]+list(set(df.columns)&set(target_variable))+Y].reset_index(drop=True)
print(new_df.iloc[0,:].isna().sum())
print(new_df.iloc[-1,:].isna().sum())

0
0


In [149]:
tmp = new_df.describe().loc['std',:][new_df.describe().loc['std',:] == 0].index
TF = new_df.describe().loc['min',:] == new_df.describe().loc['max',:]
tmp1 = new_df.describe().loc[:,TF].columns.values.tolist()

tmp = list(set(tmp) | set(tmp1))
tmp = list(set(tmp) & set(new_df.columns))

drop_lst = list(tmp)

print("before: ", new_df.shape)
print("drop_lst: ",len(list(drop_lst)))
new_df = new_df[new_df.columns.difference(drop_lst)]
print("after: ", new_df.shape)

drop_lst


before:  (76, 60)
drop_lst:  0
after:  (76, 60)


[]

In [150]:
for i in range(len(new_df.columns)):
    flg = 0
    for j in range(len(new_df)):
        if pd.isna(new_df.iloc[j,i]):
            flg += 1
        else:
            flg = 0
            
        if flg >= 4:
            print(new_df.columns[i])
            flg = 0
            break
print("4개 이상의 결측값을 가지는 컬럼 확인 ")

4개 이상의 결측값을 가지는 컬럼 확인 


In [151]:
print("결측값 확인: ", new_df.isna().sum().sum())
new_df[new_df.isna().sum()[new_df.isna().sum()>0].index].values

결측값 확인:  3


array([[5.05],
       [3.89],
       [3.73],
       [1.75],
       [2.99],
       [2.56],
       [2.97],
       [2.74],
       [3.27],
       [2.36],
       [2.92],
       [2.7 ],
       [3.96],
       [2.54],
       [3.65],
       [4.43],
       [5.66],
       [4.59],
       [5.75],
       [3.11],
       [4.68],
       [3.92],
       [4.98],
       [4.3 ],
       [ nan],
       [ nan],
       [ nan],
       [6.54],
       [5.98],
       [6.31],
       [6.23],
       [6.51],
       [6.58],
       [6.05],
       [5.75],
       [6.35],
       [5.47],
       [5.99],
       [5.49],
       [5.62],
       [5.  ],
       [4.41],
       [4.53],
       [4.71],
       [4.37],
       [4.42],
       [4.27],
       [4.15],
       [3.54],
       [3.73],
       [3.82],
       [3.9 ],
       [3.54],
       [3.93],
       [3.56],
       [3.83],
       [3.39],
       [3.69],
       [3.29],
       [3.55],
       [2.93],
       [3.3 ],
       [3.08],
       [2.84],
       [2.08],
       [2.54],
       [2.

In [152]:
# 선형 보간
print("결측값 확인(before): ", new_df.isna().sum().sum())
for _, col in enumerate(new_df.columns):
    if col != "Date_x":
        new_df[col] = new_df[col].interpolate(method='linear')

print("선형 보간 적용")
print("결측값 확인(after): ", new_df.isna().sum().sum())

결측값 확인(before):  3
선형 보간 적용
결측값 확인(after):  0


In [153]:
# 이 때, 편의를 위해 "Date_x" 변수를 맨 앞으로, Y1~Y4 변수는 맨 뒤로 위치시킴
col1 = new_df.pop("Date_x")
new_df.insert(0, col1.name, col1)

col2 = new_df.pop("close")
new_df.insert(new_df.shape[1], col2.name, col2)

col3 = new_df.pop("close_weighted")
new_df.insert(new_df.shape[1], col3.name, col3)

col4 = new_df.pop("EV_EBITDA")
new_df.insert(new_df.shape[1], col4.name, col4)

col5 = new_df.pop("PBR")
new_df.insert(new_df.shape[1], col5.name, col5)

print("new_df.shape: ",new_df.shape)


new_df.head(2)

new_df.shape:  (76, 60)


,Date_x,이자보상율,"EPS(지배, Adj.)(원/주)",매입채무회전율,재고자산회전율(비율),당기순이익(천원),유동자산(천원),매출총이익(천원),유동비율(비율),인당매출s,...,EBITDA2(천원),현금배당성향(%),자기자본비율,EBITDA2마진율(비율),CAPEX(천원),BPS증가율(YoY)(비율),close,close_weighted,EV_EBITDA,PBR
0,2003-06-01,1911.76471,0.89000,123.40000,0.38721,59.40000,602,109.70000,5.05000,0.06384,...,74.62000,0.00000,85.25403,32.02000,12.96000,-0.10253,1.80588,1.55250,13.21000,0.16000
1,2003-09-01,2035.29412,0.98000,129.60000,0.40764,65.50000,615,121.10000,3.89000,0.06273,...,83.04000,0.31908,82.44444,33.12000,15.82000,0.26789,1.80365,1.89157,17.93000,3.07000


### 파일 내보내기

In [154]:
### 최종 dataset을 csv, xlsx 파일로 out ###
new_df.to_csv(os.path.join(out_path,"ITS")+raw_file_name.split('.')[0]+"_clean_v1.3.csv", index=False, encoding='utf-8')
new_df.to_excel(os.path.join(out_path,"ITS")+raw_file_name.split('.')[0]+"_clean_v1.3.xlsx", index=False)